<a href="https://colab.research.google.com/github/glucard/Leaves-dataset/blob/disease-dataset/Colab/YOLO_dataset_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
from ultralytics import YOLO

from IPython.display import display, Image, clear_output

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.4/589.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 14.0 MB/s eta 0:00:00


#### download repo

In [2]:
!wget -O repo.zip https://github.com/glucard/Leaves-dataset/archive/refs/heads/disease-dataset.zip
!unzip repo.zip -d repo

clear_output()

In [3]:
yolo_model = YOLO('/content/repo/Leaves-dataset-disease-dataset/models/yolo_leafs_v1.pt')

In [4]:
!mv "/content/repo/Leaves-dataset-disease-dataset/data/Disease-dataset/diases_datasets_by_leaf" "/content/original_dataset"

In [5]:
import cv2
import numpy as np

IMG_SIZE = 500, 500

def load_images(path):
  
  if os.path.isdir(path):
    images_names = sorted(list(os.listdir(path)))
    images_names = [os.path.join(path, img) for img in images_names]
  else:
    images_names = [path]
  images = [cv2.imread(img) for img in images_names]

  return images

def crop_images(yolo_pred, images):
  """
  PARAMS:
  yolo_pred: yolo model output
  images: list of images presented to yolo

  returns a numpy.array
  """

  cropped_images = []
  for pred, img in zip(yolo_pred, images):
    
    bounding_boxes = [box.xyxy.cpu().numpy().astype(int)[0] for box in pred.boxes]
    if bounding_boxes:
      cropped_imgs = [img[bb[1]:bb[3], bb[0]:bb[2]] for bb in bounding_boxes]
      resized_imgs = [cv2.resize(img, IMG_SIZE, interpolation = cv2.INTER_AREA) for img in cropped_imgs]
      
      for img in resized_imgs:
        cropped_images.append(np.array(img))

  cropped_images = np.array(cropped_images)

  return cropped_images


In [6]:
import os

def check_and_create_path(path):
  if not os.path.exists(path):
    os.mkdir(path)

PATH_IN = '/content/original_dataset'
PATH_OUT = '/content/output_dataset'

check_and_create_path(PATH_OUT)

leaf_list_dir = sorted(os.listdir('/content/original_dataset'))

for leaf_dir in leaf_list_dir:
  leaf_path = os.path.join(PATH_IN, leaf_dir)
  leaf_path_out = os.path.join(PATH_OUT, leaf_dir)
  
  check_and_create_path(leaf_path_out)

  for disease_dir in os.listdir(leaf_path):
    disease_path = os.path.join(leaf_path, disease_dir)
    disease_path_out = os.path.join(leaf_path_out, disease_dir)
    
    check_and_create_path(disease_path_out)

    #predict
    pred = yolo_model.predict(disease_path)
    clear_output()
    
    #crop images
    images = load_images(disease_path)
    cropped_images = crop_images(pred, images)

    for i, img in enumerate(cropped_images):
      out_img_name = f"{leaf_dir}_{disease_dir}_{i}.jpg"
      out_img_path = os.path.join(disease_path_out, out_img_name)
      cv2.imwrite(out_img_path, img)

In [8]:
!zip -r '/content/output_dataset.zip' '/content/output_dataset'
clear_output()